In [1]:

#import libraries
import pandas as pd # this imports the pandas library which is necessary for data frames and calls that library
 # can call all the functions of this library with "pd."
import numpy as np # same for numpy
     

In [2]:
df_plnt =pd.read_excel("https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx", 
                       sheet_name="PLNT20",keep_default_na=False,header=1)
df_plnt.head(5) 

,SEQPLT20,YEAR,PSTATABB,PNAME,ORISPL,OPRNAME,OPRCODE,UTLSRVNM,UTLSRVID,SECTOR,...,PLWIPR,PLSOPR,PLGTPR,PLOFPR,PLOPPR,PLTNPR,PLTRPR,PLTHPR,PLCYPR,PLCNPR
0,1,2020,AK,7-Mile Ridge Wind Project,60814,Alaska Power and Telephone Co,219,Alaska Power and Telephone Co,219,Electric Utility,...,,,,,,,,,,
1,2,2020,AK,Agrium Kenai Nitrogen Operations,54452,Homer Electric Assn Inc,19558,Agrium US Inc,179,Industrial CHP,...,,,,,,,,,,
2,3,2020,AK,Alakanuk,57053,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,,,,,,,,,,
3,4,2020,AK,Allison Creek Hydro,58982,"Copper Valley Elec Assn, Inc",4329,"Copper Valley Elec Assn, Inc",4329,Electric Utility,...,0,0,0,0,0,0,1,0,0,1
4,5,2020,AK,Ambler,60243,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,0,0,0,0,0,1,0,0,1,0


In [3]:
#Name of columns
# 2. Select Columns that are needed for the analysis
# Plant name: PNAME
# Plant state abbreviation: PSTATABB
# Utility name: UTLSRVNM
# Plant-level sector: SECTOR
# Balancing Authority Name: BANAME
# Plant associated ISO/RTO Territory: ISORTO
# Plant capacity factor: CAPFAC
# Plant nameplate capacity: NAMEPCAP
# Plant annual net generation (MWh): PLNGENAN

# 3. Convert numeric columns to numeric, probably not necesary.
df_plnt.CAPFAC=pd.to_numeric(df_plnt.CAPFAC)
df_plnt.NAMEPCAP=pd.to_numeric(df_plnt.NAMEPCAP)
df_plnt.PLNGENAN=pd.to_numeric(df_plnt.PLNGENAN)

#Select only the attributes required, the copy function is recommended, the code 
# works without this function
df_plnt_summ = df_plnt.loc[:,['PNAME', 'PSTATABB', 'UTLSRVNM','SECTOR','BANAME', 
                              'ISORTO', 'CAPFAC','NAMEPCAP', 'PLNGENAN']].copy()

#Show first 5 plants
df_plnt_summ.head(n=5)

,PNAME,PSTATABB,UTLSRVNM,SECTOR,BANAME,ISORTO,CAPFAC,NAMEPCAP,PLNGENAN
0,7-Mile Ridge Wind Project,AK,Alaska Power and Telephone Co,Electric Utility,No balancing authority,,NaN,1.8,NaN
1,Agrium Kenai Nitrogen Operations,AK,Agrium US Inc,Industrial CHP,No balancing authority,,NaN,21.6,NaN
2,Alakanuk,AK,"Alaska Village Elec Coop, Inc",Electric Utility,No balancing authority,,0.00000,2.6,0.0
3,Allison Creek Hydro,AK,"Copper Valley Elec Assn, Inc",Electric Utility,No balancing authority,,0.36544,6.5,20808.0
4,Ambler,AK,"Alaska Village Elec Coop, Inc",Electric Utility,No balancing authority,,0.09224,1.5,1212.0


In [4]:
# 4. Filter out all the plants that had zero or blank capacity factor in 2020 
# (i.e., Column AA in PLNT20 is zero or blank).
df_plnt_summ.dropna(axis='rows', subset=['CAPFAC'],inplace=True)
df_plnt_summ = df_plnt_summ[(df_plnt_summ.CAPFAC > 0)].copy()

In [5]:

#5. Label the plants that belong to an ISO/RTO
df_plnt_summ['is_iso_rto'] = 'Yes'
df_plnt_summ.loc[df_plnt_summ.ISORTO == '', 'is_iso_rto'] = 'No'
#create pivot table by state, with Nameplate capacity and Total generation. 
# Values aggregated by plants that belong to ISO/RTO or not
pt_iso = pd.pivot_table(df_plnt_summ, index="PSTATABB", columns='is_iso_rto', 
                        values=['NAMEPCAP', 'PLNGENAN'], fill_value=0, aggfunc=['sum'])
#drop index level in columns 
pt_iso.columns = pt_iso.columns.droplevel(level =0)
pt_iso.head(n=5)

NAMEPCAP               PLNGENAN              
is_iso_rto       No      Yes            No           Yes
PSTATABB                                                
AK           3025.2      0.0  6.162056e+06  0.000000e+00
AL          35588.3      0.0  1.368420e+08  0.000000e+00
AR           2847.6  14595.3  9.468833e+06  4.517243e+07
AZ          31585.1      0.0  1.093189e+08  0.000000e+00
CA          18664.8  71669.3  3.253016e+07  1.607009e+08

In [6]:
#DEtermine in which states the total nameplate capacity of their plants mostly
# are under the jurisdiction of an ISO/RTO
pt_iso['percent_iso'] = 100*pt_iso[('NAMEPCAP','Yes')]  / (pt_iso[('NAMEPCAP','Yes')] +pt_iso[('NAMEPCAP','No')] )
# Add and initialize two columns to the pivot table.
# The columns "Number of States" will be useful later to count how many have restructured 
pt_iso['Number of States'] = 1
#The column "restructured" is to indicate for each state if it is restructured or not. Initalize in no and change later based on % of ISO/RTO
pt_iso['restructured'] = 'N'
pt_iso.loc[pt_iso.percent_iso >= 51, 'restructured'] = 'Y'

# Add capacity and generation by state 
pt_iso['Capacity in 2020 (MW)'] = pt_iso[('NAMEPCAP','Yes')] +pt_iso[('NAMEPCAP','No')]
pt_iso['Generation in 2020 (MWh)'] = pt_iso[('PLNGENAN','Yes')] +pt_iso[('PLNGENAN','No')]

#Group data by the states that are mostly reformed and not reformed
pt_iso_summary = pt_iso.groupby(['restructured']).sum()
pt_iso_summary.columns = pt_iso_summary.columns.droplevel(level =1)
pt_iso_summary.rename(index={'Y': 'RTO/ISO', 'N':'Non-RTO/ISO'}, inplace=True)
pt_iso_summary.index.name = 'Region'
pt_iso_summary[['Number of States',	'Capacity in 2020 (MW)','Generation in 2020 (MWh)']]

,Number of States,Capacity in 2020 (MW),Generation in 2020 (MWh)
Region,,,
Non-RTO/ISO,26,581620.0,1.773328e+09
RTO/ISO,26,749886.2,2.254063e+09


In [35]:
df_eia_price =pd.read_excel("Desktop/717/Assignment/Average Retail Price .xlsx", 
                       sheet_name="Average Retail Price ",index_col = 'State', usecols = [0,1,2])

In [36]:
df_eia_price.head(n=5)

,Rank,"Average Retail Price of Electricity to Residential Sector, cents/kWh"
State,,
HI,1,44.85
NH,2,30.44
MA,3,26.28
CA,4,25.49
CT,5,25.04


In [38]:
#delete Rank column
df_eia_price.drop(columns='Rank', inplace =True)

df_eia_price.head(n=5)

,"Average Retail Price of Electricity to Residential Sector, cents/kWh"
State,
HI,44.85
NH,30.44
MA,26.28
CA,25.49
CT,25.04


In [40]:
pt_restructured_state = pt_iso[['restructured']].copy()

# modify column names of dataframe, reduce one level
pt_restructured_state.columns = pt_restructured_state.columns.droplevel(level =1)
#merge dataframe of states restructuring statos with dataframe of prices.
pt_iso_price = pt_restructured_state.join(df_eia_price)


#Exclude Alaska, Hawaii, Puerto Rico and Texas 
pt_iso_price.drop(['AK', 'PR', 'HI', 'TX'], axis = 0, inplace =True)
pt_iso_price.head()

,restructured,"Average Retail Price of Electricity to Residential Sector, cents/kWh"
PSTATABB,,
AL,N,15.19
AR,Y,12.58
AZ,N,13.10
CA,Y,25.49
CO,N,14.97


In [41]:
#obtain average, minimum and maximum prices by region (RTO and non-RTO)
df_iso_price_results = pd.pivot_table(pt_iso_price, index="restructured", fill_value=0, aggfunc=['mean','min', 'max'])

#separately, obtain the number of states that belong to RTO and no-RTO regions
#The number of states will be different than in the previous question because we dropped four
#states (AK,HI, PR, and TX)
df_iso_count = pt_iso_price.groupby(['restructured'])['restructured'].count()
#Now ...
df_iso_count = df_iso_count.to_frame()

#combine information of the count of states and the 
df_iso_price_results = df_iso_count.join(df_iso_price_results)
df_iso_price_results.rename(index={'Y': 'RTO/ISO', 'N':'Non-RTO/ISO'}, inplace=True)
df_iso_price_results.index.name = 'Region'
df_iso_price_results

C:\Users\Siying Yi\AppData\Local\Temp\ipykernel_21128\3705511701.py:12: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df_iso_price_results = df_iso_count.join(df_iso_price_results)


,restructured,"(mean, Average Retail Price of Electricity to Residential Sector, cents/kWh)","(min, Average Retail Price of Electricity to Residential Sector, cents/kWh)","(max, Average Retail Price of Electricity to Residential Sector, cents/kWh)"
Region,,,,
Non-RTO/ISO,23,13.3300,10.59,15.23
RTO/ISO,25,18.4252,11.73,30.44
